In [9]:
import numpy as np
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Construct the Combined Dataset

In [2]:
# Load data from the first JSON file
data_list1 = []
with open('data/domain1_train.json', 'r') as f1:
    for line in f1:
        data_list1.append(json.loads(line))

# Load data from the second JSON file
data_list2 = []
with open('data/domain2_train.json', 'r') as f2:
    for line in f2:
        data_list2.append(json.loads(line))

# Combine the data from both lists
combined_data = data_list1 + data_list2

# Save the combined data to a new JSON file
with open('data/combined.json', 'w') as output_file:
    for item in combined_data:
        json.dump(item, output_file)
        output_file.write('\n')

print("Combined data saved to 'combined.json'")


Combined data saved to 'combined.json'


# Read the Combined Data to list and Train_Test split

In [12]:
# Read JSON data
data_list = []
with open('data/combined.json', 'r') as f:
    for line in f:
        data_list.append(json.loads(line))

# Convert to DataFrame
df = pd.DataFrame(data_list)

# Numeric sequences and corresponding labels
sequences = df['text'].to_list()
sequences = [" ".join(map(str, seq)) for seq in sequences]
labels =  df['label'].to_list() # Corresponding labels (1 for human, 0 for machine)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, random_state=42)

# Tf-Idf Preprocessing

In [13]:
# define a function for tfidf for the convenience of changing: 
# input text (in training set, validation and test); max features needed; and ngrams
def tfidf(cleanedText_train, cleanedText_test, maxFeatures = None, ngram = 1):
    '''
    This function take string form cleaned Text and process it to Tf-Idf sparse matrix
    '''
    vectorizer = TfidfVectorizer(max_features = maxFeatures,ngram_range = (ngram,ngram))
    trained_vec = vectorizer.fit(cleanedText_train)
    tfidf = trained_vec.transform(cleanedText_test)
    # check the names of terms
    #display(vectorizer.get_feature_names_out())
    return tfidf 

tfidf(sequences,sequences)

<34400x4990 sparse matrix of type '<class 'numpy.float64'>'
	with 1346067 stored elements in Compressed Sparse Row format>

In [ ]:
# Create a CountVectorizer
vectorizer = CountVectorizer()

# Convert numeric sequences into BoW-like vectors
bow_vectors = vectorizer.fit_transform([" ".join(map(str, seq)) for seq in sequences])



# Train a Naive Bayes classifier
classifier = LogisticRegression(solver='newton-cg', C=0.5, max_iter=10000)#MultinomialNB()
classifier.fit(X_train, y_train)

# Make predictions on the test data
predictions = classifier.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)